In [32]:
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import pandas as pd
from io import StringIO

class NaiveBayesClassifier:
    def __init__(self):
        self.class_priors = None
        self.feature_means = None
        self.feature_vars = None
        self.classes = None

    def train(self, X, y):
        """
        Train the Naive Bayes classifier

        Parameters:
        X: Training data features [n_samples, n_features]
        y: Training data labels [n_samples]
        """
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Initialize parameters
        self.class_priors = np.zeros(n_classes)
        self.feature_means = np.zeros((n_classes, n_features))
        self.feature_vars = np.zeros((n_classes, n_features))

        # Calculate class priors and feature statistics for each class
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[i] = X_c.shape[0] / n_samples
            self.feature_means[i, :] = X_c.mean(axis=0)
            self.feature_vars[i, :] = X_c.var(axis=0) + 1e-6  # Add small value to avoid zero variance

    def _calculate_likelihood(self, X):
        """
        Calculate likelihood of the data under each class

        Parameters:
        X: Test data features [n_samples, n_features]

        Returns:
        likelihoods: Likelihood for each sample under each class [n_samples, n_classes]
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes)
        likelihoods = np.zeros((n_samples, n_classes))

        for i in range(n_classes):
            # Gaussian probability density
            deviations = X - self.feature_means[i, :]
            exponent = -0.5 * np.sum(deviations**2 / self.feature_vars[i, :], axis=1)
            normalizer = 1 / np.sqrt((2 * np.pi) ** n_features * np.prod(self.feature_vars[i, :]))
            likelihoods[:, i] = normalizer * np.exp(exponent)

        return likelihoods

    def predict(self, X):
        """
        Predict class labels and calculate discriminant functions

        Parameters:
        X: Test data features [n_samples, n_features]

        Returns:
        predicted_classes: Predicted class labels [n_samples]
        discriminant_values: Values of discriminant functions [n_samples, n_classes]
        """
        likelihoods = self._calculate_likelihood(X)
        # Calculate posterior probabilities (discriminant functions)
        discriminant_values = likelihoods * self.class_priors

        # Normalize to get proper probabilities (optional)
        discriminant_values = discriminant_values / np.sum(discriminant_values, axis=1, keepdims=True)

        # Get predicted class (maximum posterior)
        predicted_indices = np.argmax(discriminant_values, axis=1)
        predicted_classes = self.classes[predicted_indices]

        return predicted_classes, discriminant_values

class PerceptronClassifier:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        """
        Initialize Perceptron classifier

        Parameters:
        learning_rate: Learning rate for weight updates
        n_iterations: Maximum number of iterations
        """
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.classes = None

    def _initialize_weights(self, n_features, n_classes):
        """Initialize weights and bias"""
        if n_classes == 2:
            # Binary classification: One set of weights
            self.weights = np.zeros(n_features)
            self.bias = 0
        else:
            # Multi-class: One set of weights per class
            self.weights = np.zeros((n_classes, n_features))
            self.bias = np.zeros(n_classes)

    def train(self, X, y):
        """
        Train the Perceptron classifier
        Parameters:
        X: Training data features [n_samples, n_features]
        y: Training data labels [n_samples]
        """
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Map class labels to integers starting from 0
        y_mapped = np.zeros_like(y, dtype=int)
        for i, c in enumerate(self.classes):
            y_mapped[y == c] = i

        # Initialize weights
        self._initialize_weights(n_features, n_classes)

        # Train the model
        if n_classes == 2:
            # Binary classification
            for _ in range(self.n_iterations):  # Fixed: removed asterisk
                for idx, x_i in enumerate(X):
                    # Convert class 0 to -1 for binary classification
                    y_i = 1 if y_mapped[idx] == 1 else -1

                    # Calculate activation
                    activation = np.dot(x_i, self.weights) + self.bias

                    # Update weights if misclassified
                    if y_i * activation <= 0:
                        self.weights += self.learning_rate * y_i * x_i
                        self.bias += self.learning_rate * y_i
        else:
            # Multi-class classification (one-vs-rest)
            for _ in range(self.n_iterations):  # Fixed: removed asterisk
                for idx, x_i in enumerate(X):
                    y_true = y_mapped[idx]

                    # Calculate activations for all classes
                    activations = np.dot(x_i, self.weights.T) + self.bias  # Fixed dot product orientation
                    y_pred = np.argmax(activations)

                    # Update weights if misclassified
                    if y_pred != y_true:
                        self.weights[y_true] += self.learning_rate * x_i
                        self.weights[y_pred] -= self.learning_rate * x_i
                        self.bias[y_true] += self.learning_rate
                        self.bias[y_pred] -= self.learning_rate

    def predict(self, X):
        """
        Predict class labels and calculate discriminant functions

        Parameters:
        X: Test data features [n_samples, n_features]

        Returns:
        predicted_classes: Predicted class labels [n_samples]
        discriminant_values: Values of discriminant functions [n_samples, n_classes]
        """
        n_classes = len(self.classes)

        if n_classes == 2:
            # Binary classification
            discriminant_values = np.column_stack([
                -np.dot(X, self.weights) - self.bias,  # Class 0
                np.dot(X, self.weights) + self.bias     # Class 1
            ])
            predicted_indices = np.argmax(discriminant_values, axis=1)

        else:
            # Multi-class classification
            discriminant_values = np.dot(X, self.weights.T) + self.bias
            predicted_indices = np.argmax(discriminant_values, axis=1)

        predicted_classes = self.classes[predicted_indices]

        return predicted_classes, discriminant_values


class ClassifierEvaluator:
    def __init__(self):
        pass

    def compute_confusion_matrix(self, y_true, y_pred, classes=None):
        """
        Compute confusion matrix

        Parameters:
        y_true: True class labels
        y_pred: Predicted class labels
        classes: List of class labels (if None, will be computed from the data)

        Returns:
        confusion_matrix: Confusion matrix [n_classes, n_classes]
        """
        if classes is None:
            classes = np.unique(np.concatenate((y_true, y_pred)))

        n_classes = len(classes)
        confusion_mat = np.zeros((n_classes, n_classes), dtype=int)

        for i in range(len(y_true)):
            true_idx = np.where(classes == y_true[i])[0][0]
            pred_idx = np.where(classes == y_pred[i])[0][0]
            confusion_mat[true_idx, pred_idx] += 1

        return confusion_mat, classes

    def compute_metrics(self, confusion_matrix):
        """
        Compute accuracy, precision, recall, and F1 score from confusion matrix

        Parameters:
        confusion_matrix: Confusion matrix [n_classes, n_classes]

        Returns:
        metrics_dict: Dictionary containing metrics
        """
        n_classes = confusion_matrix.shape[0]
        metrics = {}

        # Overall accuracy
        metrics['accuracy'] = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

        # Per-class metrics
        metrics['precision'] = np.zeros(n_classes)
        metrics['recall'] = np.zeros(n_classes)
        metrics['f1_score'] = np.zeros(n_classes)

        for i in range(n_classes):
            # Precision
            if np.sum(confusion_matrix[:, i]) > 0:
                metrics['precision'][i] = confusion_matrix[i, i] / np.sum(confusion_matrix[:, i])
            else:
                metrics['precision'][i] = 0

            # Recall
            if np.sum(confusion_matrix[i, :]) > 0:
                metrics['recall'][i] = confusion_matrix[i, i] / np.sum(confusion_matrix[i, :])
            else:
                metrics['recall'][i] = 0

            # F1 score
            if metrics['precision'][i] + metrics['recall'][i] > 0:
                metrics['f1_score'][i] = 2 * metrics['precision'][i] * metrics['recall'][i] / (metrics['precision'][i] + metrics['recall'][i])
            else:
                metrics['f1_score'][i] = 0

        # Macro-averaged metrics
        metrics['macro_precision'] = np.mean(metrics['precision'])
        metrics['macro_recall'] = np.mean(metrics['recall'])
        metrics['macro_f1'] = np.mean(metrics['f1_score'])

        return metrics

    def plot_confusion_matrix(self, confusion_matrix, class_names, title='Confusion Matrix'):
        """
        Plot confusion matrix

        Parameters:
        confusion_matrix: Confusion matrix [n_classes, n_classes]
        class_names: Names of classes
        title: Title of the plot
        """
        plt.figure(figsize=(8, 6))
        plt.imshow(confusion_matrix, cmap='Blues')
        plt.title(title)
        plt.colorbar()

        n_classes = len(class_names)
        tick_marks = np.arange(n_classes)
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Add text annotations
        thresh = confusion_matrix.max() / 2
        for i in range(n_classes):
            for j in range(n_classes):
                plt.text(j, i, format(confusion_matrix[i, j], 'd'),
                         ha="center", va="center",
                         color="white" if confusion_matrix[i, j] > thresh else "black")

        plt.ylabel('True Class')
        plt.xlabel('Predicted Class')
        plt.tight_layout()

    def calculate_roc_curve(self, y_true_binary, discriminant_values_positive, n_points=100):
        """
        Calculate ROC curve for binary classification

        Parameters:
        y_true_binary: Binary true class labels (0 or 1)
        discriminant_values_positive: Discriminant values for the positive class
        n_points: Number of threshold points for the ROC curve

        Returns:
        fpr_values: False positive rates
        tpr_values: True positive rates
        auc: Area under the ROC curve
        """
        # Convert labels to binary (0 or 1)
        y_true_binary = np.array(y_true_binary).astype(int)

        # Get positive and negative scores
        pos_scores = discriminant_values_positive[y_true_binary == 1]
        neg_scores = discriminant_values_positive[y_true_binary == 0]

        # Calculate ROC curve
        thresholds = np.linspace(np.min(discriminant_values_positive),
                                np.max(discriminant_values_positive), n_points)

        fpr_values = []
        tpr_values = []

        for threshold in thresholds:
            # True positive rate
            tp = np.sum(pos_scores >= threshold)
            fn = np.sum(pos_scores < threshold)
            tpr = tp / (tp + fn) if (tp + fn) > 0 else 0

            # False positive rate
            fp = np.sum(neg_scores >= threshold)
            tn = np.sum(neg_scores < threshold)
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

            fpr_values.append(fpr)
            tpr_values.append(tpr)

        # Calculate AUC using trapezoidal rule
        fpr_values = np.array(fpr_values)
        tpr_values = np.array(tpr_values)

        # Sort by increasing FPR
        sorted_indices = np.argsort(fpr_values)
        fpr_values = fpr_values[sorted_indices]
        tpr_values = tpr_values[sorted_indices]

        # Add endpoints if needed
        if fpr_values[0] > 0 or tpr_values[0] > 0:
            fpr_values = np.concatenate(([0], fpr_values))
            tpr_values = np.concatenate(([0], tpr_values))

        if fpr_values[-1] < 1 or tpr_values[-1] < 1:
            fpr_values = np.concatenate((fpr_values, [1]))
            tpr_values = np.concatenate((tpr_values, [1]))

        auc = np.trapezoid(tpr_values, fpr_values)

        return fpr_values, tpr_values, auc

    def plot_roc_curve(self, fpr_values, tpr_values, auc, label=None):
        """
        Plot ROC curve

        Parameters:
        fpr_values: False positive rates
        tpr_values: True positive rates
        auc: Area under the ROC curve
        label: Label for the curve
        """
        if label is None:
            label = f'AUC = {auc:.3f}'
        else:
            label = f'{label} (AUC = {auc:.3f})'

        plt.plot(fpr_values, tpr_values, label=label)
        plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)

def load_iris_dataset():
    """Load Iris dataset"""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    try:
        response = urlopen(url)
        data = response.read().decode('utf-8')
        df = pd.read_csv(StringIO(data), header=None, names=column_names)
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y, column_names[:-1], np.unique(y)
    except:
        print("Error loading Iris dataset from URL. Using synthetic data instead.")
        # Create synthetic Iris data if URL fails
        from sklearn.datasets import load_iris
        iris = load_iris()
        X = iris.data
        y = np.array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])[iris.target]
        return X, y, iris.feature_names, np.unique(y)

def load_breast_cancer_coimbra_dataset():
    """Load Breast Cancer Coimbra dataset"""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv"
    try:
        response = urlopen(url)
        data = response.read().decode('utf-8')
        df = pd.read_csv(StringIO(data))
        X = df.iloc[:, :-1].values  # Features
        y = df.iloc[:, -1].values   # Classification column
        # Convert class to strings for consistency
        y = np.array(['Healthy' if label == 1 else 'Patient' for label in y])
        feature_names = df.columns[:-1].tolist()
        class_names = np.unique(y)
        return X, y, feature_names, class_names
    except:
        print("Error loading Breast Cancer Coimbra dataset from URL. Generating synthetic data instead.")
        # Generate synthetic data if URL fails
        np.random.seed(42)
        n_samples = 116  # Actual dataset size
        n_features = 9   # Actual number of features
        X = np.random.randn(n_samples, n_features)
        y = np.array(['Healthy' if i < 58 else 'Patient' for i in range(n_samples)])
        feature_names = [
            'Age', 'BMI', 'Glucose', 'Insulin', 'HOMA', 'Leptin',
            'Adiponectin', 'Resistin', 'MCP.1'
        ]
        class_names = np.unique(y)
        return X, y, feature_names, class_names

def load_ionosphere_dataset():
    """Load Ionosphere dataset"""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
    column_names = [f'feature_{i}' for i in range(34)] + ['class']
    try:
        response = urlopen(url)
        data = response.read().decode('utf-8')
        df = pd.read_csv(StringIO(data), header=None, names=column_names)
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        feature_names = column_names[:-1]
        class_names = np.unique(y)
        return X, y, feature_names, class_names
    except:
        print("Error loading Ionosphere dataset from URL. Using synthetic data instead.")
        # Create synthetic ionosphere data if URL fails
        X = np.random.randn(351, 34)
        y = np.random.choice(['g', 'b'], size=351)
        feature_names = [f'feature_{i}' for i in range(34)]
        return X, y, feature_names, np.unique(y)

def load_wine_dataset():
    """Load Wine dataset"""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
    column_names = ['class', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
                   'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins',
                   'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
    try:
        response = urlopen(url)
        data = response.read().decode('utf-8')
        df = pd.read_csv(StringIO(data), header=None, names=column_names)
        X = df.iloc[:, 1:].values  # Features
        y = df.iloc[:, 0].values   # Class column
        # Convert class to strings for consistency
        y = np.array([f'Class_{int(label)}' for label in y])
        feature_names = column_names[1:]
        class_names = np.unique(y)
        return X, y, feature_names, class_names
    except:
        print("Error loading Wine dataset from URL. Using synthetic data instead.")
        # Create synthetic wine data if URL fails
        from sklearn.datasets import load_wine
        wine = load_wine()
        X = wine.data
        y = np.array([f'Class_{i+1}' for i in wine.target])
        return X, y, wine.feature_names, np.unique(y)

def split_data(X, y, train_ratio=0.7, random_state=42):
    """
    Split data into training and testing sets with similar class distributions

    Parameters:
    X: Features
    y: Labels
    train_ratio: Ratio of training data
    random_state: Random seed for reproducibility

    Returns:
    X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)

    # Get unique classes and their indices
    classes = np.unique(y)
    indices_per_class = [np.where(y == c)[0] for c in classes]

    # Split indices for each class
    train_indices = []
    test_indices = []

    for class_indices in indices_per_class:
        np.random.shuffle(class_indices)
        n_train = int(len(class_indices) * train_ratio)

        train_indices.extend(class_indices[:n_train])
        test_indices.extend(class_indices[n_train:])

    # Get train/test splits
    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test

def cross_validation(X, y, n_folds=5, random_state=42):
    """
    Perform k-fold cross-validation

    Parameters:
    X: Features
    y: Labels
    classifier: Classifier object with train and predict methods
    n_folds: Number of folds
    random_state: Random seed for reproducibility

    Returns:
    X_train, X_test, y_train, y_test
    #mean_accuracy: Mean accuracy across folds
    #std_accuracy: Standard deviation of accuracy across folds
    """
    np.random.seed(random_state)

    # Get indices for each class
    classes = np.unique(y)
    indices_per_class = [np.where(y == c)[0] for c in classes]

    # Create stratified folds
    fold_indices = [[] for _ in range(n_folds)]

    for class_indices in indices_per_class:
        np.random.shuffle(class_indices)
        # Split class indices into n_folds parts
        fold_size = len(class_indices) // n_folds

        for fold_idx in range(n_folds):
            start_idx = fold_idx * fold_size
            end_idx = (fold_idx + 1) * fold_size if fold_idx < n_folds - 1 else len(class_indices)
            fold_indices[fold_idx].extend(class_indices[start_idx:end_idx])

    # Run cross-validation
    accuracies = []

    for test_fold in range(n_folds):
        # Create train/test split
        test_indices = fold_indices[test_fold]
        train_indices = []
        for fold_idx in range(n_folds):
            if fold_idx != test_fold:
                train_indices.extend(fold_indices[fold_idx])

        X_train = X[train_indices]
        y_train = y[train_indices]
        X_test = X[test_indices]
        y_test = y[test_indices]

    return X_train, X_test, y_train, y_test

    """
        # Train and evaluate classifier
        classifier.train(X_train, y_train)
        y_pred, _ = classifier.predict(X_test)

        # Calculate accuracy
        accuracy = np.mean(y_pred == y_test)
        accuracies.append(accuracy)

    return np.mean(accuracies), np.std(accuracies)
    """

def normalize_data(X_train, X_test):
    """
    Normalize data using min-max scaling based on training data

    Parameters:
    X_train: Training features
    X_test: Testing features

    Returns:
    X_train_norm: Normalized training features
    X_test_norm: Normalized testing features
    """
    # Convert boolean values to integers before normalization
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Calculate min and max values from training data
    min_vals = np.min(X_train, axis=0)
    max_vals = np.max(X_train, axis=0)
    range_vals = max_vals - min_vals

    # Avoid division by zero
    range_vals[range_vals == 0] = 1

    # Normalize
    X_train_norm = (X_train - min_vals) / range_vals
    X_test_norm = (X_test - min_vals) / range_vals

    return X_train_norm, X_test_norm

import numpy as np

def standardiize_data(X_train, X_test):
    """
    Normalize data using Z-score standardization based on training data

    Parameters:
    X_train: Training features
    X_test: Testing features

    Returns:
    X_train_norm: Standardized training features
    X_test_norm: Standardized testing features
    """
    # Convert boolean values to floats before standardization
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Calculate mean and standard deviation from training data
    mean_vals = np.mean(X_train, axis=0)
    std_vals = np.std(X_train, axis=0)

    # Avoid division by zero
    std_vals[std_vals == 0] = 1

    # Standardize
    X_train_norm = (X_train - mean_vals) / std_vals
    X_test_norm = (X_test - mean_vals) / std_vals

    return X_train_norm, X_test_norm


def main():
    print("Classifier Evaluation Program")
    print("-----------------------------")

    # Load datasets
    print("\nLoading datasets...")


    # Breast Cancer Coimbra dataset (binary)
    X_bc, y_bc, feature_names_bc, class_names_bc = load_breast_cancer_coimbra_dataset()
    print(f"Breast Cancer Coimbra dataset loaded: {X_bc.shape[0]} samples, {X_bc.shape[1]} features, {len(class_names_bc)} classes")


    # Ionosphere (binary)
    X_ion, y_ion, feature_names_ion, class_names_ion = load_ionosphere_dataset()
    print(f"Ionosphere dataset loaded: {X_ion.shape[0]} samples, {X_ion.shape[1]} features, {len(class_names_ion)} classes")


    # Iris dataset (multi-class)
    X_iris, y_iris, feature_names_iris, class_names_iris = load_iris_dataset()
    print(f"Iris dataset loaded: {X_iris.shape[0]} samples, {X_iris.shape[1]} features, {len(class_names_iris)} classes")

    # Wine dataset (multi-class)
    X_wine, y_wine, feature_names_wine, class_names_wine = load_wine_dataset()
    print(f"Wine dataset loaded: {X_wine.shape[0]} samples, {X_wine.shape[1]} features, {len(class_names_wine)} classes")

    # Process datasets
    datasets = [
        {
            'name': 'Breast Cancer Coimbra',
            'X': X_bc,
            'y': y_bc,
            'feature_names': feature_names_bc,
            'class_names': class_names_bc,
            'binary': True
        },
        {
            'name': 'Ionosphere',
            'X': X_ion,
            'y': y_ion,
            'feature_names': feature_names_ion,
            'class_names': class_names_ion,
            'binary': True
        },
        {
            'name': 'Iris',
            'X': X_iris,
            'y': y_iris,
            'feature_names': feature_names_iris,
            'class_names': class_names_iris,
            'binary': False
        },
        {
            'name': 'Wine',
            'X': X_wine,
            'y': y_wine,
            'feature_names': feature_names_wine,
            'class_names': class_names_wine,
            'binary': False
        }
    ]


    # Create classifier instances
    classifiers = [
        {
            'name': 'Naive Bayes',
            'model': NaiveBayesClassifier()
        },
        {
            'name': 'Perceptron',
            'model': PerceptronClassifier(learning_rate=0.01, n_iterations=1000)
        }
    ]

    # Create evaluator
    evaluator = ClassifierEvaluator()

    # Store ROC results
    roc_results = []


    # Process each dataset
    for dataset in datasets:
        print(f"\n\n===== Dataset: {dataset['name']} =====")

        for i in range(0,3):
          # Split data
          X_train, X_test, y_train, y_test = split_data(dataset['X'], dataset['y'], train_ratio=0.7)
          print(f"Data split: \n {X_train.shape[0]} training samples, {X_test.shape[0]} testing samples")


        #for k  in range(5,11):
          # Cross-validation
          #X_train, X_test, y_train, y_test = cross_validation(dataset['X'], dataset['y'], n_folds=k)
          #print(f"\nCross-Validation ({k}-fold): \n{X_train.shape[0]} training samples, {X_test.shape[0]} testing samples")

          # Without normalization
          if (i==0):
            print(f"without normalization: \n")
            X_train_norm = X_train
            X_test_norm = X_test
          # Normalize data
          elif(i==1):
            print(f"with min-max: \n")
            X_train_norm, X_test_norm = normalize_data(X_train, X_test)
          else:
            print(f"with z-score: \n")
            X_train_norm, X_test_norm = standardiize_data(X_train, X_test)

          # Evaluate each classifier
          for clf_info in classifiers:
              classifier = clf_info['model']
              clf_name = clf_info['name']

              print(f"\n----- Classifier: {clf_name} -----")

              # Train classifier
              classifier.train(X_train_norm, y_train)

              # Test classifier
              y_pred, discriminant_values = classifier.predict(X_test_norm)

              # Compute confusion matrix
              conf_matrix, classes = evaluator.compute_confusion_matrix(y_test, y_pred, dataset['class_names'])

              # Compute metrics
              metrics = evaluator.compute_metrics(conf_matrix)

              # Display results
              print(f"Accuracy: {metrics['accuracy']:.4f}")
              print(f"Macro-Precision: {metrics['macro_precision']:.4f}")
              print(f"Macro-Recall: {metrics['macro_recall']:.4f}")
              print(f"Macro-F1: {metrics['macro_f1']:.4f}")

              # Display confusion matrix
              print("\nConfusion Matrix:")
              print(" " * 12, end="")
              for c in classes:
                  print(f"{c[:7]:>10}", end="")
              print()

              for i, c in enumerate(classes):
                  print(f"{c[:10]:>10} |", end="")
                  for j in range(len(classes)):
                      print(f"{conf_matrix[i, j]:>10}", end="")
                  print()


              # Plot confusion matrix
              evaluator.plot_confusion_matrix(conf_matrix, classes, title=f'{clf_name} Confusion Matrix - {dataset["name"]}')

              # For binary datasets, compute ROC curve
              if dataset['binary']:
                  # Convert class labels to binary (0 or 1)
                  binary_labels = np.zeros(len(y_test))
                  positive_class = dataset['class_names'][0]  # First class is positive
                  binary_labels[y_test == positive_class] = 1

                  # Get discriminant values for positive class
                  if len(discriminant_values.shape) > 1 and discriminant_values.shape[1] > 1:
                      # Multi-output discriminant (use first class)
                      disc_positive = discriminant_values[:, 0]
                  else:
                      # Single output discriminant
                      disc_positive = discriminant_values

                  # Calculate and plot ROC curve
                  fpr, tpr, auc = evaluator.calculate_roc_curve(binary_labels, disc_positive)
                  print(f"\nAUC: {auc:.4f}")

                  # Store ROC results for later comparison
                  if not hasattr(main, 'roc_results'):
                      main.roc_results = []

                  main.roc_results.append({
                      'dataset': dataset['name'],
                      'classifier': clf_name,
                      'fpr': fpr,
                      'tpr': tpr,
                      'auc': auc
                  })

              # Plot ROC curves for binary dataset
              if hasattr(main, 'roc_results'):
                  plt.figure(figsize=(10, 8))

                  for result in main.roc_results:
                      label = f"{result['classifier']} - {result['dataset']}"
                      evaluator.plot_roc_curve(result['fpr'], result['tpr'], result['auc'], label=label)

                  plt.title('ROC Curves Comparison')
                  plt.tight_layout()

              # Show all plots
              plt.show()


if __name__ == "__main__":
    main()

Classifier Evaluation Program
-----------------------------

Loading datasets...
Breast Cancer Coimbra dataset loaded: 116 samples, 9 features, 2 classes


===== Dataset: Breast Cancer Coimbra =====
Data split: 
 80 training samples, 36 testing samples
without normalization: 


----- Classifier: Naive Bayes -----
Accuracy: 0.6667
Macro-Precision: 0.7385
Macro-Recall: 0.6937
Macro-F1: 0.6571

Confusion Matrix:
               Healthy   Patient
   Healthy |        15         1
   Patient |        11         9

----- Classifier: Perceptron -----
Accuracy: 0.5556
Macro-Precision: 0.2778
Macro-Recall: 0.5000
Macro-F1: 0.3571

Confusion Matrix:
               Healthy   Patient
   Healthy |         0        16
   Patient |         0        20
Data split: 
 80 training samples, 36 testing samples
with min-max: 


----- Classifier: Naive Bayes -----
Accuracy: 0.6667
Macro-Precision: 0.7385
Macro-Recall: 0.6937
Macro-F1: 0.6571

Confusion Matrix:
               Healthy   Patient
   Healthy |     